In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 60

In [ ]:
#utils.py is where all our custorm functions live is we set an autoreload on it.
%aimport utils 
from utils import *

# The data

In [ ]:
df_all = pd.read_excel('all data v3.xlsx', 'iLab data.txt', index_col=None, na_values=['NA'])

### Preparing a test sample
Let's first use a particular session as a test case. We extract only the data relevant to that case

In [ ]:
df_test = prepare_session(df_all,'L-2567b17a:120eda25685:-8000')
df_gaps = prepare_session(df_all,'L-10f11766:120ecd4f63a:-8000')

# The plot

In [ ]:
colors = {"Cases":"white",
         "Now try working on this new example":"#252525",
         "intuition": "#969696",
         "Single value": "red",
         'Central tendency': "#33a02c",
         'Distance': "#6a3d9a",
#          'Average': "#6a3d9a",
#          'Sum': "#1f78b4",
          'Count gaps': "red",
          'Range': "#1f78b4",
         'Count all': "#ff7f00",
         'evaluation steps': "#525252",
          'submit': "#525252",
          "Build":'black',
          "deleteAll":'black',
        }
column_to_use = {"Now try working on this new example":"Feedback Text",
                 "intuition": "Selection",
                 "submit": "Selection",
                 "evaluation": "Selection",
                 "checkIntuition": "Selection",
                 'Median': 'Cleaned method 1',
                 'Average': 'Cleaned method 1',
                 'Sum': 'Cleaned method 1',
                 'Count all': 'Cleaned method 1',
                 "deleteAll":"Selection",
                }

function_to_use = {"Cases":case_usage,
                   "Single value":single_value_usage,
                   "Central tendency":central_tendency_usage,
                   "Range":range_usage,
                   "Distance":distance_usage,
                   "Count gaps":count_gaps_usage,
                   "Build": build_events,
                   "evaluation steps":evaluation_steps_usage,}

to_plot = ["Cases","intuition",'Single value','Central tendency',"Count all","Count gaps",'Range',"Distance","Build","deleteAll","submit","evaluation steps"]

In [ ]:
def plot(df,to_plot,colors, column_to_use, function_to_use):
    fig = plt.figure(figsize=(18,9))
    ax = plt.subplot()
    spacing =10
    pos = 0
    max_time = 0
    actions = list(reversed(to_plot))
    black = '#252525'
    for i,action in enumerate(actions):
        if action == "Cases":
            cases = all_cases(df)
            for case,coords in all_cases(df).items():
                left = [float(x) for x in case[0].split(" ")]
                right = [float(x) for x in case[1].split(" ")]
                ymax = max(max(left),max(right))
                ymin = min(min(left),min(right))
                Xl = [coords[0]+30]*len(left)
                Yl = [l/(ymax)*(spacing-2.5)+1+pos for l in left]
                Xr = [coords[0]+30+20]*len(right)
                Yr = [r/(ymax)*(spacing-2.5)+1+pos for r in right]
                ax.plot(Xl,Yl,'.',color="darkgrey",markersize=10)
                ax.plot(Xr,Yr,'.',color="darkgrey",markersize=10)
        if action in column_to_use.keys():
            action_use = action_usage(df,column_to_use[action],action)
        else:
            action_use = function_to_use[action](df)
        if action_use:
            max_time = max(max_time,sum(action_use[-1]))
            ax.broken_barh(action_use,(pos,spacing),facecolors=colors[action],alpha=1,linewidth=0)
        pos += spacing

#     #Add text of method
#     action_use = action_usage(df,column_to_use['submit'],'submit')
#     methods = []
#     i = -3
#     for (x,y) in action_use: 
#         ax.text(x,i,'M',fontsize=12)
#         i= i-3
#         if i <-13:
#             i = -3



#     #Add horizontal bar
#     ax.broken_barh([(0,ax.get_xlim()[1])],((len(actions))*spacing,spacing),facecolors='white',alpha=1,linewidth=0)

    #Add new case bar
    new_case = "Now try working on this new example"
    action_use = action_usage(df,column_to_use[new_case],new_case)
    case_use = [(x,10) for (x,y) in action_use]+[(0,10)]
    max_time = max(max_time,sum(action_use[-1]))
    ax.broken_barh(case_use,(0,(len(actions))*spacing),facecolors="white",alpha=1,linewidth=0)

    #Add levaels
    ax.set_xlabel('minutes in activity',fontsize=13)
    ax.set_xticks(range(0,int(max_time),60))
    ax.set_xticklabels([str(x/60)+''if x in range(0,int(max_time),60*5) else "" for x in range(0,int(max_time),60)],fontsize=13)
    ax.set_yticks(range(spacing/2,len(actions)*spacing,spacing))
    ax.set_yticklabels([a.capitalize() for a in actions],fontsize=15)
    ax.grid(True)
    plt.show()

### Session with gap counting

In [ ]:
plot(df_gaps,to_plot,colors, column_to_use, function_to_use)

In [ ]:
plot(df_gaps,to_plot,colors, column_to_use, function_to_use)

### Session with range and extrapolated range

In [ ]:
plot(df_test,to_plot,colors, column_to_use, function_to_use)

### TO DO
#### Ordered in some kind of general priority (in terms of need of feedback and desired feature)
1. central tendency (average median sum) - add "choose 2 4 5 7" in regex
* add Other category
* Show the method that eventually “cracks” the cases (succeeds to solve and move on)
* fix visual component of build, delete and submit
* clean visual - embellish
* run on all 9 student pairs

#### What can invention be? (Rename as Other)  - Two options:
1. Lots of differernt pre-set possibilities 
    * Anytime they use addition within a step?
    * Anytime they use multiplication
    * Use a combination of central tendency
    * Use a combination of methods (range and mean)
2. Any method that isn't other methods
    * after removing chuncks that fit our methods, is there anything left?
    * or
    * anything that has more than two steps subtracting time coords of methods.
